In [2]:
import open3d as o3d
import numpy as np
import laspy
import matplotlib.pyplot as plt
import imageio
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
# Load the LAS file
las_file = r"C:\Users\gedsloov\OneDrive - UGent\UGent\PhD\05_Research\02_ICOS\02_Data\ICOS_BRA_for_presentation\ICOS_BRA 0.100 m.las"
las = laspy.read(las_file)

In [11]:
las

<LasData(1.4, point fmt: <PointFormat(3, 6 bytes of extra dims)>, 23246698 points, 2 vlrs)>

In [12]:
# Extract coordinates
points = np.vstack((las.x, las.y, las.z)).T  # X, Y, Z

In [ ]:
# Extract RGB colors (Normalize to 0-1 for Open3D)
colors = np.vstack((las.red, las.green, las.blue)).T / 65535  # LAS stores RGB in 16-bit


In [13]:
# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
#pcd.colors = o3d.utility.Vector3dVector(colors)

In [14]:
# Downsampling (if necessary)
if len(points) > 1_000_000:  # Reduce if the file is too large
    pcd = pcd.voxel_down_sample(voxel_size=0.05)  # Adjust voxel size as needed

In [15]:
# Set up rotation angles
num_frames = 36  # Number of frames for full rotation
rotation_axis = [0, 0, 1]  # Rotate around Z-axis
output_folder = "rotation_frames"
os.makedirs(output_folder, exist_ok=True)

In [16]:
# Generate frames
for i in range(num_frames):
    angle = (i / num_frames) * 360  # Rotate 360° over frames
    R = pcd.get_rotation_matrix_from_axis_angle(np.radians(angle) * np.array(rotation_axis))
    pcd_rotated = pcd.rotate(R, center=pcd.get_center())

    # Render and save frame
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.add_geometry(pcd_rotated)
    vis.poll_events()
    vis.update_renderer()
    
    # Capture and save image
    image_path = f"{output_folder}/frame_{i:03d}.png"
    vis.capture_screen_image(image_path)
    vis.destroy_window()

In [17]:
# Create GIF from saved frames
frame_files = sorted([f"{output_folder}/{f}" for f in os.listdir(output_folder) if f.endswith(".png")])
frames = [imageio.imread(f) for f in frame_files]
gif_path = "rotating_pointcloud.gif"
imageio.mimsave(gif_path, frames, fps=12)  # Adjust FPS for speed

print(f"GIF saved as {gif_path}")

C:\Users\gedsloov\AppData\Local\Temp\ipykernel_20684\3191174520.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames = [imageio.imread(f) for f in frame_files]


GIF saved as rotating_pointcloud.gif
